In [1]:
#We need to construct a global dataframe object where we can query everything

In [2]:
import requests
import json
from urllib.parse import urlparse, urlsplit
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
def generate_output_dict(url_list):
    output = {}
    for url in url_list:
        parsed_url = urlparse(url)
        key = parsed_url.path.split('/')[-1]
        output[key] = {
            "url": url,
            "data": None
        }
    return output

#Adding the invoice-lines and payment-allocations
url_list = [
    "https://hackathon.syftanalytics.com/api/contacts",
    "https://hackathon.syftanalytics.com/api/item",
    "https://hackathon.syftanalytics.com/api/invoice",
    "https://hackathon.syftanalytics.com/api/payment",
    "https://hackathon.syftanalytics.com/api/invoice-lines"
]

output = generate_output_dict(url_list)
headers = {'x-api-key': 'e6506999-8738-4866-a13f-2a2cfb14ba99'}

# Iterating through the URLs and getting the data back
for key in output:
    response = requests.get(output[key]['url'], headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        json_data = response.json()["data"]
        # Add the data of each URL to the output dictionary
        output[key]['data'] = json_data
        #print(f"Data for {key}:")
        #print(json.dumps(json_data, indent=4))
        
    else:
        print(key)
        print(f"Failed to get data for {key}: {response.content}")

In [4]:
list(output.keys())

['contacts', 'item', 'invoice', 'payment', 'invoice-lines']

In [5]:
#Creating dataframes for each of the elements we want to analyse
contacts_df = pd.DataFrame(output["contacts"]["data"]) 
item_df = pd.DataFrame(output["item"]["data"])
invoice_df = pd.DataFrame(output["invoice"]["data"])
payment_df = pd.DataFrame(output["payment"]["data"])
invoice_info_df = pd.DataFrame(output["invoice-lines"]["data"])

Database amalgamation

In [6]:
#Joining the invoice info to the invoice df => their common columns are id and invoice_id
#creating a column called invoice_id in invoice df to match with the info df
invoice_df["invoice_id"] = invoice_df["id"]

#doing the match based on the common columns
invoice_merged = pd.merge(invoice_df, invoice_info_df, on='invoice_id', how='outer')#using outer so that the one's that do not match are kept

In [7]:
invoice_merged.head()

,id_x,issue_date,due_date,paid_date,paid,contact_id,total_x,amount_due,exchange_rate,currency,is_sale,invoice_id,id_y,description,item_code,total_y,quantity
0,90381d5a-9348-4208-915c-60b082378370,2021-01-31T00:00:00.000Z,2021-02-06T00:00:00.000Z,2021-02-06T00:00:00.000Z,True,8e81fbc4-4f33-4ae9-bf5e-a2415372e77b,174.800000,0.0,1.0000,GBP,True,90381d5a-9348-4208-915c-60b082378370,a90834f9-90b5-4342-9cb6-f8d45b35fd7e,Rental income,None,174.800000,1
1,e4f53ac4-3aa8-4332-9082-c6551e09d90b,2021-02-23T00:00:00.000Z,2021-02-28T00:00:00.000Z,None,False,e0857d09-69cc-4608-83b8-7e2083e678a3,10000.000000,0.0,1.0000,GBP,True,e4f53ac4-3aa8-4332-9082-c6551e09d90b,0163c3f0-97e7-4309-9056-0d1d745db656,Purchase of building,None,10000.000000,1
2,3dec381c-7af5-436d-8b35-2d3a69b95f89,2021-02-28T00:00:00.000Z,2021-03-06T00:00:00.000Z,2021-03-06T00:00:00.000Z,True,8e81fbc4-4f33-4ae9-bf5e-a2415372e77b,174.800000,0.0,1.0000,GBP,True,3dec381c-7af5-436d-8b35-2d3a69b95f89,3554bf7d-bd10-449c-b05a-5a5e478bab73,Rental income,None,174.800000,1
3,aca26d49-abcb-4320-ac50-e0e613e34393,2021-03-07T00:00:00.000Z,2021-03-25T00:00:00.000Z,2022-04-05T00:00:00.000Z,True,cf2b417f-352e-486f-a5e0-a32cbbfa0826,848.346016,0.0,1.9317,NZD,True,aca26d49-abcb-4320-ac50-e0e613e34393,6299f20b-baf1-46f6-9c87-531ef2a91cda,Baking Lessons,None,848.346016,1
4,d1c5eee6-9b87-46ed-a153-0a344f242675,2021-03-14T00:00:00.000Z,2021-04-30T00:00:00.000Z,2021-04-30T00:00:00.000Z,True,d7d2b5fc-32e7-4011-94a4-2c8aa0ff9e4f,1725.000000,0.0,1.0000,GBP,True,d1c5eee6-9b87-46ed-a153-0a344f242675,3e2ee452-d7c7-4bf5-95ba-c0aae26815a6,Cinderella Cupcakes,CIN001,1725.000000,100


In [8]:
#Fixing the columns that have been renamed
invoice_merged = invoice_merged.rename(columns={"id_x":"id_invoice", "id_y":"id_invoice_info", "total_x":"total"})

In [9]:
#dropping the total_y because it is a repeat of the total
invoice_merged.drop(["total_y"], axis=1, inplace=True)

In [10]:
invoice_merged.head()

,id_invoice,issue_date,due_date,paid_date,paid,contact_id,total,amount_due,exchange_rate,currency,is_sale,invoice_id,id_invoice_info,description,item_code,quantity
0,90381d5a-9348-4208-915c-60b082378370,2021-01-31T00:00:00.000Z,2021-02-06T00:00:00.000Z,2021-02-06T00:00:00.000Z,True,8e81fbc4-4f33-4ae9-bf5e-a2415372e77b,174.800000,0.0,1.0000,GBP,True,90381d5a-9348-4208-915c-60b082378370,a90834f9-90b5-4342-9cb6-f8d45b35fd7e,Rental income,None,1
1,e4f53ac4-3aa8-4332-9082-c6551e09d90b,2021-02-23T00:00:00.000Z,2021-02-28T00:00:00.000Z,None,False,e0857d09-69cc-4608-83b8-7e2083e678a3,10000.000000,0.0,1.0000,GBP,True,e4f53ac4-3aa8-4332-9082-c6551e09d90b,0163c3f0-97e7-4309-9056-0d1d745db656,Purchase of building,None,1
2,3dec381c-7af5-436d-8b35-2d3a69b95f89,2021-02-28T00:00:00.000Z,2021-03-06T00:00:00.000Z,2021-03-06T00:00:00.000Z,True,8e81fbc4-4f33-4ae9-bf5e-a2415372e77b,174.800000,0.0,1.0000,GBP,True,3dec381c-7af5-436d-8b35-2d3a69b95f89,3554bf7d-bd10-449c-b05a-5a5e478bab73,Rental income,None,1
3,aca26d49-abcb-4320-ac50-e0e613e34393,2021-03-07T00:00:00.000Z,2021-03-25T00:00:00.000Z,2022-04-05T00:00:00.000Z,True,cf2b417f-352e-486f-a5e0-a32cbbfa0826,848.346016,0.0,1.9317,NZD,True,aca26d49-abcb-4320-ac50-e0e613e34393,6299f20b-baf1-46f6-9c87-531ef2a91cda,Baking Lessons,None,1
4,d1c5eee6-9b87-46ed-a153-0a344f242675,2021-03-14T00:00:00.000Z,2021-04-30T00:00:00.000Z,2021-04-30T00:00:00.000Z,True,d7d2b5fc-32e7-4011-94a4-2c8aa0ff9e4f,1725.000000,0.0,1.0000,GBP,True,d1c5eee6-9b87-46ed-a153-0a344f242675,3e2ee452-d7c7-4bf5-95ba-c0aae26815a6,Cinderella Cupcakes,CIN001,100


In [11]:
#Now merging the contacts in the common column is contacts
#creating the common column in the contacts_df
contacts_df["contact_id"] = contacts_df["id"]
contacts_merged = pd.merge(invoice_merged, contacts_df, on='contact_id', how='outer')

In [12]:
contacts_merged.tail(20)

,id_invoice,issue_date,due_date,paid_date,paid,contact_id,total,amount_due,exchange_rate,currency,...,id_invoice_info,description,item_code,quantity,id,name,is_supplier,is_customer,email,phone
466,ea705f83-049c-4c70-9fc4-61b42938a2cc,2022-04-11T00:00:00.000Z,2022-04-27T00:00:00.000Z,2022-04-27T00:00:00.000Z,True,763190a0-8e54-4082-ad28-4b25aaf3b31d,150000.0,0.0,1.0,GBP,...,33b6f9a6-1a80-4eb5-a7f2-be2d3ca16386,Zoo Chocolates,CHO001,1000.0,763190a0-8e54-4082-ad28-4b25aaf3b31d,Dawn Bellwether,True,False,None,None
467,21879939-6fe7-4055-a929-10a9288fa1ea,2022-05-19T00:00:00.000Z,2022-05-31T00:00:00.000Z,2022-05-31T00:00:00.000Z,True,763190a0-8e54-4082-ad28-4b25aaf3b31d,15000.0,0.0,1.0,GBP,...,040b078c-a03e-4c0d-8194-8ec1b54df01f,Zoo Chocolates,CHO001,100.0,763190a0-8e54-4082-ad28-4b25aaf3b31d,Dawn Bellwether,True,False,None,None
468,295fa554-f9f5-42d9-aba1-dc353b279a50,2022-09-04T00:00:00.000Z,2022-10-31T00:00:00.000Z,2022-10-31T00:00:00.000Z,True,763190a0-8e54-4082-ad28-4b25aaf3b31d,11500.0,0.0,1.0,GBP,...,0c4cdf90-6b40-4b8a-99a1-a9d45930154d,Capitalised,None,1.0,763190a0-8e54-4082-ad28-4b25aaf3b31d,Dawn Bellwether,True,False,None,None
469,16b5fe38-5924-48a3-9059-b1e93181226e,2022-09-15T00:00:00.000Z,2022-09-30T00:00:00.000Z,2022-09-30T00:00:00.000Z,True,763190a0-8e54-4082-ad28-4b25aaf3b31d,67500.0,0.0,1.0,GBP,...,6cf110a8-1471-42cd-a897-14350831e360,Zoo Chocolates,CHO001,500.0,763190a0-8e54-4082-ad28-4b25aaf3b31d,Dawn Bellwether,True,False,None,None
470,05029b84-355f-446d-906e-b7a6df2cef8b,2022-04-11T00:00:00.000Z,2022-05-25T00:00:00.000Z,2022-04-30T00:00:00.000Z,True,2ef70219-eed4-4d6b-a152-5885e7024b49,230.0,0.0,1.0,GBP,...,6be7a5a5-6358-4d2c-82b7-b854966c2926,Computer Additions,None,2.0,2ef70219-eed4-4d6b-a152-5885e7024b49,Shere Khan,True,False,None,None
471,33e2667c-db63-4ea5-9d0b-3e71eac98b11,2023-06-14T00:00:00.000Z,2023-06-14T00:00:00.000Z,2023-06-14T00:00:00.000Z,True,2ef70219-eed4-4d6b-a152-5885e7024b49,2000.0,0.0,1.0,GBP,...,a80b51eb-cd6e-412b-9289-8656df9f8e6e,Staff Party,None,1.0,2ef70219-eed4-4d6b-a152-5885e7024b49,Shere Khan,True,False,None,None
472,b175a000-ac55-4dd6-b02f-c521d9d49ff9,2022-05-01T00:00:00.000Z,2022-05-30T00:00:00.000Z,2022-05-30T00:00:00.000Z,True,1e6f39b7-de54-4df7-aa60-ce090fb54705,2500.0,0.0,1.0,GBP,...,e3f40d12-2db8-407b-8ed7-dba2e62493da,Year end Party - Entertainment,None,1.0,1e6f39b7-de54-4df7-aa60-ce090fb54705,Elle Elves,True,False,None,None
473,110c0c16-fb1e-48a8-8e9a-285e491cb686,2022-05-10T00:00:00.000Z,2022-06-08T00:00:00.000Z,2022-08-15T00:00:00.000Z,True,c7060a16-02ba-4029-ab85-4369caa9f18a,287.5,0.0,1.0,GBP,...,0d448b6d-b297-4efc-9715-e78e9c01ba9f,Additional Office Machine,None,1.0,c7060a16-02ba-4029-ab85-4369caa9f18a,Gastonia,True,False,None,None
474,63942613-8f19-4bb0-b592-7fdaeb5d8d83,2022-12-05T00:00:00.000Z,2023-01-04T00:00:00.000Z,2023-01-04T00:00:00.000Z,True,c7060a16-02ba-4029-ab85-4369caa9f18a,5750.0,0.0,1.0,GBP,...,350f161f-819b-409c-9c50-da0e132758dc,Consulting Fees,None,1.0,c7060a16-02ba-4029-ab85-4369caa9f18a,Gastonia,True,False,None,None
475,2926e3aa-83b9-475e-a080-8d0956ec4ad1,2023-01-05T00:00:00.000Z,2023-02-04T00:00:00.000Z,2023-02-04T00:00:00.000Z,True,c7060a16-02ba-4029-ab85-4369caa9f18a,5750.0,0.0,1.0,GBP,...,9acfda56-6efb-4f4b-b8f4-f3177b92c92d,Consulting Fees,None,1.0,c7060a16-02ba-4029-ab85-4369caa9f18a,Gastonia,True,False,None,None


In [13]:
contacts_merged.columns

Index(['id_invoice', 'issue_date', 'due_date', 'paid_date', 'paid',
       'contact_id', 'total', 'amount_due', 'exchange_rate', 'currency',
       'is_sale', 'invoice_id', 'id_invoice_info', 'description', 'item_code',
       'quantity', 'id', 'name', 'is_supplier', 'is_customer', 'email',
       'phone'],
      dtype='object')

In [14]:
contacts_merged.to_csv("./data/merged.csv")